Data & code: https://www.kaggle.com/nareshbhat/health-care-data-set-on-heart-attack-possibility

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#df = pd.read_csv("../input/health-care-data-set-on-heart-attack-possibility/heart.csv")
df = pd.read_csv("../input/health-care-data-set-on-heart-attack-possibility/heart.csv")

<b>About data set<br></b>
This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them. In particular, the Cleveland database is the only one that has been used by ML researchers to
this date.The "target" field refers to the presence of heart disease in the patient. It is integer valued 0 = no/less chance of heart attack and 1 = more chance of heart attack<br>
Attribute Information<br>
1) age<br>
2) sex<br>
3) chest pain type (4 values)<br>
4) resting blood pressure<br>
5) serum cholestoral in mg/dl<br>
6)fasting blood sugar > 120 mg/dl<br>
7) resting electrocardiographic results (values 0,1,2)<br>
8) maximum heart rate achieved<br>
9) exercise induced angina<br>
10) oldpeak = ST depression induced by exercise relative to rest<br>
11)the slope of the peak exercise ST segment<br>
12) number of major vessels (0-3) colored by flourosopy<br>
13) thal: 0 = normal; 1 = fixed defect; 2 = reversable defect<br>
14) target: 0= less chance of heart attack 1= more chance of heart attack<br>

# 데이터 파악하기

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.shape

In [ ]:
df.describe()

# EDA(Exploratory Data Analysis)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

fig,axes=plt.subplots(nrows=1,ncols=1,figsize=(10,5))
sns.countplot(df.target)

In [ ]:
fig,axes=plt.subplots(nrows=1,ncols=1,figsize=(15,10))
sns.distplot(df['age'],hist=True,kde=True,rug=False,label='age',norm_hist=True)

In [ ]:
df.columns

In [ ]:
corr = df.corr(method = 'pearson')
corr

In [ ]:
colormap = plt.cm.PuBu 
plt.figure(figsize=(15, 10)) 
plt.title("Person Correlation of Features", y = 1.05, size = 15) 
sns.heatmap(corr.astype(float).corr(), linecolor = "white", cmap = colormap, annot = True)

In [ ]:
import plotly.express as px
px.bar(df, x= 'age' , y='target', color='sex' , title= 'heart attack patoents age range and sex',
      labels = { 'output': 'Number of patients', 'Age': 'Age od patient'})

# 학습모델 생성 및 예측하기

In [ ]:
X = df.drop(columns= ['target'])
y= df['target']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# 데이터 정규화 [0, 1]

transformer = MinMaxScaler()
transformer.fit(X)
X = transformer.transform(X)
X

In [ ]:
from sklearn.model_selection import train_test_split
x_test,x_train,y_test,y_train = train_test_split(X,y,test_size = 0.2,random_state = 123)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)

In [ ]:
y_pred = lr.predict( x_test)
y_pred_proba = lr.predict_proba(x_test)[:, 1]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix

cm=confusion_matrix(y_test, y_pred)
fig, ax = plot_confusion_matrix(conf_mat=cm)
plt.rcParams['font.size'] = 20
#(conf_mat=multiclass,colorbar=True, show_absolute=False, show_normed=True, class_names=class_names)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss


[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)

print('Train/Test split results:')
print(lr.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(lr.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(lr.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

idx = np.min(np.where(tpr > 0.95)) # index of the first threshold for which the sensibility > 0.95

plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='coral', label='ROC curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot([0,fpr[idx]], [tpr[idx],tpr[idx]], 'k--', color='blue')
plt.plot([fpr[idx],fpr[idx]], [0,tpr[idx]], 'k--', color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=10)
plt.ylabel('True Positive Rate (recall)', fontsize=10)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()